#### Packages

In [39]:
import os
import re
import sys
import time  # 強制等待
import requests
import pandas as pd  # 載入 pandas
from bs4 import BeautifulSoup

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options  # 設定 driver 的行為
from selenium.webdriver.support.ui import Select  # 選擇＂下拉式選單＂
from selenium.webdriver.common.keys import Keys  # 鍵盤操作
from selenium.common.exceptions import NoSuchElementException, TimeoutException  # 載入常見錯誤
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities  # 更改載入策略
from selenium.webdriver.support.ui import WebDriverWait  # 等待機制
from selenium.webdriver.support import expected_conditions as EC  # 預期事件
from selenium.webdriver.common.by import By  # 找尋元素的方法

#### 環境設置

In [40]:
chrome_options = webdriver.ChromeOptions()
chrome_options.binary_location = os.environ.get('GOOGLE_CHROME_BIN')
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--headless')

# chrome_options.add_argument('disable-dev-shm-usage')
# chrome_options.add_argument('--no-sandbox')
chrome_capabilities = DesiredCapabilities.CHROME
chrome_capabilities['pageLoadStrategy'] = 'eager'  # 頁面加載策略：HTML 解析成 DOM
def get_chrome():
    return webdriver.Chrome(options=chrome_options,
                            desired_capabilities=chrome_capabilities)

In [41]:

def wait_for_element_clickable(driver, element_position, waiting_time=5, by=By.LINK_TEXT):
    function = sys._getframe().f_code.co_name
    alert_execution_report(function)
    print_all_arguments(locals())
    try:
        node_off()
        element = WebDriverWait(driver, waiting_time).until(EC.element_to_be_clickable((by, element_position)))
    except Exception as e:
        alert_exception_report(function, e)
        return False
    else:
        return element

def accurately_find_table_and_read_it(driver, table_position, table_index=0):
    function = sys._getframe().f_code.co_name
    alert_execution_report(function)
    print_all_arguments(locals())
    try:
        node_off()
        if not wait_for_element_present(driver, table_position):
            alert_exception_report(function, 'not found table')
            return
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        table_innerHTML = soup.select(table_position)[table_index]
        tgt = pd.read_html(str(table_innerHTML), encoding='utf-8')[0]
        # tgt['圖書館'], tgt['連結'] = org, driver.current_url
    except Exception as e:
        alert_exception_report(function, e)
        return
    else:
        return tgt

def get_all_tgt_urls(driver, link_text):
    function = sys._getframe().f_code.co_name
    alert_execution_report(function)
    print_all_arguments(locals())
    
    tgt_urls = []

    anchors = driver.find_elements_by_link_text(link_text)
    for anchor in anchors:
        tgt_urls.append(anchor.get_attribute('href'))
    
    return tgt_urls

### 開始爬蟲

- 上市公司代號

In [52]:
id_Ltd = ["1101", "1102", "1103", "1104", "1108", "1109", "1110", "1201", "1203", '1210']

In [ ]:
# id_Ltd = [1101, 1102, 1103, 1104, 1108, 1109, 1110, 1201, 1203, 1210, 
#           1213, 1215, 1216, 1217 ,1218 ,1219, 1220, 1225, 1227, 1229, 
#           1231, 1232, 1233, 1234, 1235, 1236, 1256, 1301, 1303, 1304, 
#           1305, 1307, 1308, 1309, 1310, 1312, 1313, 1314, 1315, 1316, 
#           1319, 1321, 1323, 1324, 1325, 1326, 1339, 1341, 1342, 1402, 
#           1409, 1410, 1413, 1414, 1416, 1417, 1418, 1419, 1423, 1432, 
#           1434, 1435, 1436, 1437, 1438, 1439, 1440, 1441, 1442, 1443, 
#           1444, 1445, 1446, 1447, 1449, 1451, 1452, 1453, 1454, 1455, 
#           1456, 1457, 1459, 1460, 1463, 1464, 1465, 1466, 1467, 1468, 
#           1470, 1471, 1472, 1473, 1474, 1475, 1476, 1477, 1503, 1504, 
#           1506, 1512, 1513, 1514, 1515, 1516, 1517, 1519, 1521, 1522, 
#           1524, 1525, 1526, 1527, 1528, 1529, 1530, 1531, 1532, 1533, 
#           1535, 1536, 1537, 1538, 1539, 1540, 1541, 1558, 1560, 1568, 
#           1582, 1583, 1587, 1589, 1590, 1597, 1598, 1603, 1604, 1605, 
#           1608, 1609, 1611, 1612, 1614, 1615, 1616, 1617, 1618, 1701, 
#           1702, 1707, 1708, 1709, 1710, 1711, 1712, 1713, 1714, 1717, 
#           1718, 1720, 1721, 1722, 1723, 1725, 1726, 1727, 1730, 1731, 
#           1732, 1733, 1734, 1735, 1736, 1737, 1752, 1760, 1762, 1773, 
#           1776, 1783, 1786, 1789, 1795, 1802, 1805, 1806, 1808, 1809, 
#           1810, 1817, 1903, 1904, 1905, 1906, 1907, 1909, 2002, 2006, 
#           2007, 2008, 2009, 2010, 2012, 2013, 2014, 2015, 2017, 2020,
#           2022, 2023, 2024, 2025, 2027, 2028, 2029, 2030, 2031, 2032, 
#           2033, 2034, 2038, 2049, 2059, 2062, 2069, 2101, 2102, 2103, 
#           2104, 2105, 2106, 2107, 2108, 2109, 2114, 2115, 2201, 2204, 
#           2206, 2207, 2208, 2211, 2227, 2228, 2231, 2233, 2239, 2241, 
#           2247, 2250, 2301, 2303, 2305, 2308, 2312, 2313, 2314, 2316, 
#           2317, 2321, 2323, 2324, 2327, 2328, 2329, 2330, 2331, 2332, 
#           2337, 2338, 2340, 2342, 2344, 2345, 2347, 2348, 2349, 2351, 
#           2352, 2354, 2355, 2356, 2357, 2358, 2359, 2360, 2362, 2363, 
#           2364, 2365, 2367, 2368, 2369, 2371, 2373, 2374, 2375, 2376, 
#           2377, 2379, 2380, 2382, 2383, 2385, 2387, 2388, 2390, 2392, 
#           2393
# ]

In [53]:
# Set up Selenium WebDriver
driver = webdriver.Chrome(ChromeDriverManager().install())
url = 'https://mops.twse.com.tw/mops/web/t100sb12'

driver.get(url)
# 申報年度
year_input = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.NAME, "year")))
year_input.clear()
year_input.send_keys("111")

# List to store data for all companies
all_company_data = []

for company_id in id_Ltd:
    try:
        print("Processing company ID:", company_id)
        
        # 公司代號或簡稱
        id_input = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "co_id")))   
        id_input.clear()
        id_input.send_keys(str(company_id))
        
        search_button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "/html/body/center/table/tbody/tr/td/div[4]/table/tbody/tr/td/div/table/tbody/tr/td[3]/div/div[3]/form/table/tbody/tr/td[4]/table/tbody/tr/td[2]/div/div/input")))
        search_button.click()

        # Wait for the search results to load
        time.sleep(1)  # Adjust sleep time as needed
        
        # Find the table containing the data (assuming the table reloads with new data)
        center_table = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//*[@id='table01']/center/table")))

        # Extract data from the table
        company_data = {}
        rows = center_table.find_elements(By.TAG_NAME, "tr")
        for row in rows:
            header = row.find_element(By.TAG_NAME, "th").text.strip()  # Extract the header
            try:
                content_cell = row.find_element(By.TAG_NAME, "td")  # Find the <td> cell in the row
                content = content_cell.find_element(By.TAG_NAME, "pre").text.strip()  # Extract the content from <pre>
            except NoSuchElementException:
                content = "N/A"  # Placeholder value if <td> or <pre> not found
            company_data[header] = content
    
        # Append the data for this company to the list
        all_company_data.append(company_data)
        
        id_input.clear() 
        
        print("Data collected for company ID:", company_id)
        
    except Exception as e:
        print("Error processing company ID:", company_id)
        print("Error message:", str(e))
                  
driver.quit()

# Print the collected data for all companies
for idx, company_data in enumerate(all_company_data, start=1):
    print("Company", idx, "data:", company_data)


Processing company ID: 1101
Data collected for company ID: 1101
Processing company ID: 1102
Data collected for company ID: 1102
Processing company ID: 1103
Data collected for company ID: 1103
Processing company ID: 1104
Data collected for company ID: 1104
Processing company ID: 1108
Data collected for company ID: 1108
Processing company ID: 1109
Data collected for company ID: 1109
Processing company ID: 1110
Data collected for company ID: 1110
Processing company ID: 1201
Data collected for company ID: 1201
Processing company ID: 1203
Data collected for company ID: 1203
Processing company ID: 1210
Data collected for company ID: 1210
Company 1 data: {'項目': 'N/A', '員工福利政\n策': '台泥自成立以來一向重視員工權益及福利，針對有關勞資關係之制度與措施，皆遵照並落實勞工法令規定辦理，以確保公司員工權益保障之透明度。透過勞資會議、部門會議、員工大會、意見箱等管道，建立員工溝通之機制，了解員工需求，達到有效溝通及維持和諧勞資關係。本公司提除供優質工作場域及具市場競爭力之薪酬福利制度，更積極於培育人才；對於員工各項福利、進修、訓練、退休制度等，皆以優於或符合法令之前提下進行規劃執行，致力於打造勞資和諧之工作環境。', '員工權益維\n護措施': '1)台泥提供具競爭力的薪資與獎酬，並規劃完整職等、職級制度，不因性別、種族、宗教、政治立場、婚姻狀況而有差別待遇，以保障員工權益，並吸引及保留人才。\n2)本公司恪守全球

#### 整理爬到的資料

In [57]:
def clean_company_data(company_data):
    """
    Remove "\n" characters from the keys and values in the given company data dictionary.

    Args:
    - company_data (dict): A dictionary containing company data where values may contain "\n" characters.

    Returns:
    - dict: A new dictionary with "\n" characters removed from the values.
    """
    cleaned_data = {key.replace("\n", ""): value.replace("\n", "") for key, value in company_data.items()}
    return cleaned_data

cleaned_all_company_data = [clean_company_data(company_data) for company_data in all_company_data]

# Print the cleaned data for all companies
for idx, cleaned_company_data in enumerate(cleaned_all_company_data, start=1):
    print("Company", idx, "cleaned data:", cleaned_company_data)

Company 1 cleaned data: {'項目': 'N/A', '員工福利政策': '台泥自成立以來一向重視員工權益及福利，針對有關勞資關係之制度與措施，皆遵照並落實勞工法令規定辦理，以確保公司員工權益保障之透明度。透過勞資會議、部門會議、員工大會、意見箱等管道，建立員工溝通之機制，了解員工需求，達到有效溝通及維持和諧勞資關係。本公司提除供優質工作場域及具市場競爭力之薪酬福利制度，更積極於培育人才；對於員工各項福利、進修、訓練、退休制度等，皆以優於或符合法令之前提下進行規劃執行，致力於打造勞資和諧之工作環境。', '員工權益維護措施': '1)台泥提供具競爭力的薪資與獎酬，並規劃完整職等、職級制度，不因性別、種族、宗教、政治立場、婚姻狀況而有差別待遇，以保障員工權益，並吸引及保留人才。2)本公司恪守全球各營運據點所在地之勞動相關法規，保障員工之合法權益，並遵循《聯合國全球盟約》、《聯合國世界人權宣言》及《國際勞工組織工作基本原則與權利宣言》等各項國際人權公約所揭櫫之人權保護精神與基本原則，體現尊重與保護人權之責任。3)本公司依法設置「勞工退休準備金監督委員會」定期提撥退休金提存於台灣銀行股份有限公司(原中央信託局)，且定期召開委員會，審核退休金提撥及運用情形，以保障同仁之權益。另針對選擇採用勞工退休金新制之員工，依法令規定每月提繳至員工在勞保局之個人帳戶,以維護員工權益。4)為維護友善安心的工作環境，設立員工安心平台及意見信箱，透過設置多元溝通管道，建立良好的溝通及諮詢交流，使員工之想法與意見得以及時獲得反應與處理，促進勞資雙向溝通。', '勞資糾紛情形': '截至110年度及年報刊印日止,本公司並無因勞資糾紛;遭受重大損失。', '履行社會責任': '1)本公司依勞基法及相關法令規定進行員工管理規則之制定。2)依「性別工作平等法」設立性騷擾申訴專用信箱，提供員工申訴管道。3)設置員工餐廳及休閒健身房，定期進行辦公大樓安檢相關作業，以提供安全及健康之工作環境。4)不定期舉行職能教育訓練，提昇員工職涯競爭力。5)訂定及定期執行台泥人權盡職調查程序，透過人權議題風險辨識與評估、依評估結果設計管理措施及風險減緩措施，進行改善及後續追蹤，以利有效降低人權風險之影響與衝擊。6)本公司對社會公益參與不遺餘力，與社福團體合作，物資募捐、關懷偏鄉弱勢學童及獨居長輩居家改

In [60]:
print("Cleaned data for all companies:")
print(cleaned_all_company_data)

Cleaned data for all companies:
[{'項目': 'N/A', '員工福利政策': '台泥自成立以來一向重視員工權益及福利，針對有關勞資關係之制度與措施，皆遵照並落實勞工法令規定辦理，以確保公司員工權益保障之透明度。透過勞資會議、部門會議、員工大會、意見箱等管道，建立員工溝通之機制，了解員工需求，達到有效溝通及維持和諧勞資關係。本公司提除供優質工作場域及具市場競爭力之薪酬福利制度，更積極於培育人才；對於員工各項福利、進修、訓練、退休制度等，皆以優於或符合法令之前提下進行規劃執行，致力於打造勞資和諧之工作環境。', '員工權益維護措施': '1)台泥提供具競爭力的薪資與獎酬，並規劃完整職等、職級制度，不因性別、種族、宗教、政治立場、婚姻狀況而有差別待遇，以保障員工權益，並吸引及保留人才。2)本公司恪守全球各營運據點所在地之勞動相關法規，保障員工之合法權益，並遵循《聯合國全球盟約》、《聯合國世界人權宣言》及《國際勞工組織工作基本原則與權利宣言》等各項國際人權公約所揭櫫之人權保護精神與基本原則，體現尊重與保護人權之責任。3)本公司依法設置「勞工退休準備金監督委員會」定期提撥退休金提存於台灣銀行股份有限公司(原中央信託局)，且定期召開委員會，審核退休金提撥及運用情形，以保障同仁之權益。另針對選擇採用勞工退休金新制之員工，依法令規定每月提繳至員工在勞保局之個人帳戶,以維護員工權益。4)為維護友善安心的工作環境，設立員工安心平台及意見信箱，透過設置多元溝通管道，建立良好的溝通及諮詢交流，使員工之想法與意見得以及時獲得反應與處理，促進勞資雙向溝通。', '勞資糾紛情形': '截至110年度及年報刊印日止,本公司並無因勞資糾紛;遭受重大損失。', '履行社會責任': '1)本公司依勞基法及相關法令規定進行員工管理規則之制定。2)依「性別工作平等法」設立性騷擾申訴專用信箱，提供員工申訴管道。3)設置員工餐廳及休閒健身房，定期進行辦公大樓安檢相關作業，以提供安全及健康之工作環境。4)不定期舉行職能教育訓練，提昇員工職涯競爭力。5)訂定及定期執行台泥人權盡職調查程序，透過人權議題風險辨識與評估、依評估結果設計管理措施及風險減緩措施，進行改善及後續追蹤，以利有效降低人權風險之影響與衝擊。6)本公司對社會公益參與不遺餘力，與社福團體合作，物資募捐、關懷偏鄉弱勢學

#### 分析公司的表面功夫(?)

In [67]:
speech_keywords = ["溝通", "意見信箱", "意見箱", "員工意見", "言論自由", "資料隱私"]
train_keywords = ["外訓", "外部訓練", "員工培訓", "教育訓練", "進修"]

# Check for keywords and create DataFrame
data = []
for idx, company_data in enumerate(cleaned_all_company_data, start=1):
    free_speech = 1 if any(word in value for value in company_data.values() for word in speech_keywords) else 0
    staff_train = 1 if any(word in value for value in company_data.values() for word in train_keywords) else 0
    data.append({'代號': id_Ltd[idx - 1], 'free_speech': free_speech, 'staff_train': staff_train})

# Create DataFrame
df = pd.DataFrame(data)

In [68]:
df

,代號,free_speech,staff_train
0,1101,1,1
1,1102,1,1
2,1103,1,1
3,1104,1,1
4,1108,1,1
5,1109,1,1
6,1110,1,1
7,1201,1,1
8,1203,0,1
9,1210,1,1
